# 편의시설 데이터 처리 방법

아래의 코드는 맥도날드, 롯데리아, 버거킹, 스타벅스, CGV, 메가박스, 롯데리아, CU, GS25, 세븐일레븐의 데이터를 가공한 코드입니다.

각각의 편의시설마다 동을 기준으로 갯수를 카운트해 점수를 부여합니다

In [1]:
import pandas as pd

### 1. 기존 데이터 불러오기

먼저 서울 법정동 주소목록(최종).txt와 맥도날드.txt 파일을 불러옵니다.

이제 법정동 주소는 korea_data, 맥도날드는 mcdonald_data 입니다.

In [2]:
korea_data = pd.read_csv('../data_gdr/1-2. 서울 법정동 주소목록(최종).txt', sep="\t")
korea_data["gu"] = korea_data.apply(lambda x:x["법정동명"].split()[1] if len(x["법정동명"].split())==3 else "",axis=1)
korea_data["dong"] = korea_data.apply(lambda x:x["법정동명"].split()[2] if len(x["법정동명"].split())==3 else "",axis=1)
korea_data.rename(columns={"법정동명":"address"}, inplace = True)
korea_data

,address,gu,dong
0,서울특별시,,
1,서울특별시 종로구,,
2,서울특별시 종로구 청운동,종로구,청운동
3,서울특별시 종로구 신교동,종로구,신교동
4,서울특별시 종로구 궁정동,종로구,궁정동
...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동
489,서울특별시 강동구 암사동,강동구,암사동
490,서울특별시 강동구 성내동,강동구,성내동
491,서울특별시 강동구 천호동,강동구,천호동


In [3]:
mcdonald_data = pd.read_csv('./패스트푸드점/맥도날드.txt')
mcdonald_data['gu'] = mcdonald_data.apply(lambda x:x["address"].split()[1], axis=1)
mcdonald_data['dong'] = mcdonald_data.apply(lambda x:x["address"].split()[2], axis=1)
mcdonald_data

,Unnamed: 0,title,address,roadAddress,mapx,mapy,category,description,telephone,link,gu,dong
0,0,<b>맥도날드</b> 종로3가점,서울특별시 종로구 종로3가 10,서울특별시 종로구 종로 115,311020,552520,양식>햄버거,NaN,NaN,http://www.mcdonalds.co.kr/,종로구,종로3가
1,1,<b>맥도날드</b> 대학로점,서울특별시 종로구 동숭동 1-34,서울특별시 종로구 대학로 130 맥도널드,312061,553878,양식>햄버거,NaN,NaN,http://www.mcdonalds.co.kr/,종로구,동숭동
2,2,<b>맥도날드</b> 서울동묘역점,서울특별시 종로구 창신동 290,서울특별시 종로구 종로 339,313201,552784,양식>햄버거,NaN,NaN,NaN,종로구,창신동
3,3,<b>맥도날드</b> 성균관대점,서울특별시 종로구 명륜2가 197-1,서울특별시 종로구 창경궁로 241-1 김해만치과의원,311778,553935,양식>햄버거,NaN,NaN,NaN,종로구,명륜2가
4,4,<b>맥도날드</b> 안국역점,서울특별시 종로구 안국동 164,서울특별시 종로구 율곡로 45,310488,553163,양식>햄버거,NaN,NaN,http://www.mcdonalds.co.kr/,종로구,안국동
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,<b>맥도날드</b> 위례신도시점,서울특별시 송파구 장지동 896,서울특별시 송파구 위례광장로 290 위례중앙푸르지오2단지,324364,541785,양식>햄버거,NaN,NaN,NaN,송파구,장지동
96,96,<b>맥도날드</b> 강동구청점,서울특별시 강동구 성내동 539-2,서울특별시 강동구 성내로 15 삼육빌딩,322651,547929,양식>햄버거,NaN,NaN,http://www.mcdonalds.co.kr/,강동구,성내동
97,97,<b>맥도날드</b>서울둔촌DT점,서울특별시 강동구 둔촌동 517-2,서울특별시 강동구 양재대로 1382,323948,547817,양식>햄버거,NaN,NaN,NaN,강동구,둔촌동
98,98,<b>맥도날드</b> 굽은다리역DT,서울특별시 강동구 천호동 31-10,서울특별시 강동구 양재대로 1587,324475,549805,양식>햄버거,NaN,NaN,http://www.mcdonalds.co.kr/,강동구,천호동


### 2. 맥도날드 갯수만 담긴 데이터 만들기

mcdonald_data를 dong을 기준으로 groupby 시켜준 후 

새로운 new_mac으로 지정해 줍니다.

new_mac에 동마다 count 한 맥도날드 갯수를 추가해줍니다.

이제 new_mac은 맥도날드 갯수를 가진 데이터가 됩니다

In [4]:
new_mac = mcdonald_data.groupby("dong")
new_mac = pd.DataFrame(new_mac["address"].count())
new_mac.columns = ["mac_count"]
new_mac

,mac_count
dong,
가락동,1
가산동,3
갈현2동,1
고척동,1
공릉동,1
...,...
홍제동,1
화곡동,1
화양동,1


### 3. korea_data와  new_mac 합치기

이제 맥도날드 갯수가 담긴 new_mac과  korea_data를 합칩니다.

In [5]:
korea_data_count = pd.merge(korea_data,new_mac,on="dong" ,how="left").fillna(0)
korea_data_count["mac_count"] = korea_data_count["mac_count"].apply(lambda x:int(x))
korea_data_count

,address,gu,dong,mac_count
0,서울특별시,,,0
1,서울특별시 종로구,,,0
2,서울특별시 종로구 청운동,종로구,청운동,0
3,서울특별시 종로구 신교동,종로구,신교동,0
4,서울특별시 종로구 궁정동,종로구,궁정동,0
...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,1
489,서울특별시 강동구 암사동,강동구,암사동,0
490,서울특별시 강동구 성내동,강동구,성내동,1
491,서울특별시 강동구 천호동,강동구,천호동,1


### 4. 맥도날드 갯수를 점수로 환산한 데이터 만들기

new_mac 데이터를 바탕으로 맥도날드 점수가 담긴 데이터를 만듭니다.

이제 맥도날드 점수가 담긴 데이터는 new_mac_score 입니다

In [6]:
max_mac = new_mac['mac_count'].max()
new_mac_score = new_mac.loc[:, ('mac_count')]
new_mac_score= pd.DataFrame(new_mac_score) / max_mac * 10
new_mac_score.columns = ['mac_score']
new_mac_score

,mac_score
dong,
가락동,3.333333
가산동,10.000000
갈현2동,3.333333
고척동,3.333333
공릉동,3.333333
...,...
홍제동,3.333333
화곡동,3.333333
화양동,3.333333


### 5. new_mac_score와 korea_data 합치기

korea_data 에 맥도날드 점수를 합쳐서 

새로운 korea_data_score 데이터를 생성합니다.

In [7]:
korea_data_score = pd.merge(korea_data,new_mac_score,on="dong" ,how="left").fillna(0)
korea_data_score

,address,gu,dong,mac_score
0,서울특별시,,,0.000000
1,서울특별시 종로구,,,0.000000
2,서울특별시 종로구 청운동,종로구,청운동,0.000000
3,서울특별시 종로구 신교동,종로구,신교동,0.000000
4,서울특별시 종로구 궁정동,종로구,궁정동,0.000000
...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,3.333333
489,서울특별시 강동구 암사동,강동구,암사동,0.000000
490,서울특별시 강동구 성내동,강동구,성내동,3.333333
491,서울특별시 강동구 천호동,강동구,천호동,3.333333


### 1. 기존 데이터 불러오기
롯데리아.txt 파일을 불러옵니다.

이제 법정동 주소는 korea_data, 롯데리아는 lotteria_data 입니다.

In [8]:
# 롯데리아 데이터 생성
lotteria_data = pd.read_csv('./패스트푸드점/롯데리아.txt')
lotteria_data['gu'] = lotteria_data.apply(lambda x:x["address"].split()[1], axis=1)
lotteria_data['dong'] = lotteria_data.apply(lambda x:x["address"].split()[2], axis=1)
lotteria_data

,Unnamed: 0,title,address,roadAddress,mapx,mapy,category,description,telephone,link,gu,dong
0,0,<b>롯데리아</b> 종각역점,서울특별시 종로구 종로2가 84-8,서울특별시 종로구 종로 68,310531,552458,양식>햄버거,NaN,NaN,http://www.lotteria.com/,종로구,종로2가
1,1,<b>롯데리아</b>혜화점,서울특별시 종로구 혜화동 111-7,서울특별시 종로구 창경궁로 273,311939,554201,양식>햄버거,NaN,02-741-1956,http://www.lotteria.com/,종로구,혜화동
2,2,<b>롯데리아</b> 동묘역점,서울특별시 종로구 창신동 326-6,서울특별시 종로구 종로 336,313193,552729,양식>햄버거,NaN,02-764-5333,http://www.lotteria.com/,종로구,창신동
3,3,<b>롯데리아</b> 동대문광장점,서울특별시 종로구 창신동 444-14,서울특별시 종로구 종로 286 이스턴호텔,312772,552529,양식>햄버거,NaN,02-742-5747,http://www.lotteria.com/,종로구,창신동
4,4,<b>롯데리아</b> 동대문역사문화공원역점,서울특별시 중구 을지로6가 18-130,서울특별시 중구 을지로 255,312446,552042,양식>햄버거,NaN,02-2265-0374,http://www.lotteria.com/,중구,을지로6가
...,...,...,...,...,...,...,...,...,...,...,...,...
183,183,<b>롯데리아</b> 암사역점,서울특별시 강동구 암사동 500-3,서울특별시 강동구 올림픽로 792,323190,550306,양식>햄버거,NaN,02-426-4800,http://www.lotteria.com/,강동구,암사동
184,184,<b>롯데리아</b> 고덕역점,서울특별시 강동구 명일동 47-12 명일씨티코아 1층,서울특별시 강동구 동남로73길 26 고덕복합빌딩,325507,550451,양식>햄버거,NaN,02-426-9900,http://www.lotteria.com/,강동구,명일동
185,185,<b>롯데리아</b> 강일점,서울특별시 강동구 강일동 676,서울특별시 강동구 아리수로93길 33-9 강일프라자,327156,551834,양식>햄버거,NaN,02-427-0707,http://www.lotteria.com/,강동구,강일동
186,186,<b>롯데리아</b> 강동구청DT점,서울특별시 강동구 성내동 319-1,서울특별시 강동구 올림픽로 556 코알라주유소,322532,548047,양식>햄버거,NaN,02-483-1241,NaN,강동구,성내동


### 2. 롯데리아 갯수만 담긴 데이터 만들기

lotteria_data를 dong을 기준으로 groupby 시켜준 후 

새로운 new_lotte으로 지정해 줍니다.

new_lotte에 동마다 count 한 롯데리아 갯수를 추가해줍니다.

이제 new_lotte은 롯데리아 갯수를 가진 데이터가 됩니다

In [9]:
new_lotte = lotteria_data.groupby("dong")
new_lotte = pd.DataFrame(new_lotte["address"].count())
new_lotte.columns = ["lot_count"]
new_lotte

,lot_count
dong,
가산동,1
갈월동,1
강일동,1
개봉동,1
개포동,1
...,...
화곡1동,2
화곡동,1
화양동,1


### 3. korea_data와  new_lotte 합치기

이제 롯데리아 갯수가 담긴 new_lotte와 위의 korea_data를 합칩니다.
기존 korea_data에 맥도날드 갯수가 합쳐진 데이터프레임과 합쳐집니다.

In [10]:
korea_data_count

,address,gu,dong,mac_count
0,서울특별시,,,0
1,서울특별시 종로구,,,0
2,서울특별시 종로구 청운동,종로구,청운동,0
3,서울특별시 종로구 신교동,종로구,신교동,0
4,서울특별시 종로구 궁정동,종로구,궁정동,0
...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,1
489,서울특별시 강동구 암사동,강동구,암사동,0
490,서울특별시 강동구 성내동,강동구,성내동,1
491,서울특별시 강동구 천호동,강동구,천호동,1


In [11]:
korea_data_count = pd.merge(korea_data_count,new_lotte,on="dong" ,how="left").fillna(0)
korea_data_count["lot_count"] = korea_data_count["lot_count"].apply(lambda x:int(x))
korea_data_count

,address,gu,dong,mac_count,lot_count
0,서울특별시,,,0,0
1,서울특별시 종로구,,,0,0
2,서울특별시 종로구 청운동,종로구,청운동,0,0
3,서울특별시 종로구 신교동,종로구,신교동,0,0
4,서울특별시 종로구 궁정동,종로구,궁정동,0,0
...,...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,1,0
489,서울특별시 강동구 암사동,강동구,암사동,0,1
490,서울특별시 강동구 성내동,강동구,성내동,1,1
491,서울특별시 강동구 천호동,강동구,천호동,1,1


### 4. 롯데리아 갯수를 점수로 환산한 데이터 만들기

new_lotte 데이터를 바탕으로 롯데리아 점수가 담긴 데이터를 만듭니다.

이제 롯데리아 점수가 담긴 데이터는 new_lot_score 입니다

In [12]:
max_lot = new_lotte['lot_count'].max()
new_lot_score = new_lotte.loc[:, ('lot_count')]
new_lot_score= pd.DataFrame(new_lot_score) / max_lot * 10
new_lot_score.columns = ['lot_score']
new_lot_score

,lot_score
dong,
가산동,1.25
갈월동,1.25
강일동,1.25
개봉동,1.25
개포동,1.25
...,...
화곡1동,2.50
화곡동,1.25
화양동,1.25


### 5. new_lot_score와 korea_data_score 합치기

korea_data_score 에 롯데리아 점수를 합쳐서 

새로운 korea_data_score 데이터를 갱신합니다.

In [13]:
korea_data_score = pd.merge(korea_data_score,new_lot_score,on="dong" ,how="left").fillna(0)
korea_data_score

,address,gu,dong,mac_score,lot_score
0,서울특별시,,,0.000000,0.00
1,서울특별시 종로구,,,0.000000,0.00
2,서울특별시 종로구 청운동,종로구,청운동,0.000000,0.00
3,서울특별시 종로구 신교동,종로구,신교동,0.000000,0.00
4,서울특별시 종로구 궁정동,종로구,궁정동,0.000000,0.00
...,...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,3.333333,0.00
489,서울특별시 강동구 암사동,강동구,암사동,0.000000,1.25
490,서울특별시 강동구 성내동,강동구,성내동,3.333333,1.25
491,서울특별시 강동구 천호동,강동구,천호동,3.333333,1.25


### 1. 기존 데이터 불러오기
버거킹.txt 파일을 불러옵니다.

이제 법정동 주소는 korea_data, 버거킹은 burgerking_data 입니다.

In [14]:
# 버거킹 데이터 생성
burgerking_data = pd.read_csv('./패스트푸드점/버거킹.txt')

burgerking_data['gu'] = burgerking_data.apply(lambda x:x["address"].split()[1], axis=1)
burgerking_data['dong'] = burgerking_data.apply(lambda x:x["address"].split()[2], axis=1)
burgerking_data

,Unnamed: 0,title,address,roadAddress,mapx,mapy,category,description,telephone,link,gu,dong
0,0,<b>버거킹</b> 종로점,서울특별시 종로구 종로2가 56-23,서울특별시 종로구 종로 94,310810,552454,양식>햄버거,NaN,02-2285-4838,http://www.burgerking.co.kr/,종로구,종로2가
1,1,<b>버거킹</b> 종로구청점,서울특별시 종로구 수송동 68-1,서울특별시 종로구 삼봉로 57 호수빌딩,310142,552743,양식>햄버거,NaN,02-722-0238,http://www.burgerking.co.kr/,종로구,수송동
2,2,<b>버거킹</b> 혜화역점,서울특별시 종로구 혜화동 179-4,서울특별시 종로구 동숭길 146,312071,553941,양식>햄버거,NaN,NaN,NaN,종로구,혜화동
3,3,<b>버거킹</b> 시청역점,서울특별시 중구 태평로2가 330-2,서울특별시 중구 서소문로 136,309735,551767,양식>햄버거,NaN,02-777-0332,http://www.burgerking.co.kr/,중구,태평로2가
4,4,<b>버거킹</b> 약수역점,서울특별시 중구 신당동 370-64,서울특별시 중구 다산로 142,312877,550914,양식>햄버거,NaN,NaN,NaN,중구,신당동
...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,<b>버거킹</b> 천호역점,서울특별시 강동구 성내동 62-4 태승빌딩 1층,서울특별시 강동구 천호대로 1012,322810,548774,양식>햄버거,NaN,02-473-0331,http://www.burgerking.co.kr/,강동구,성내동
110,110,<b>버거킹</b> 길동사거리점,서울특별시 강동구 길동 429,서울특별시 강동구 양재대로 1440,324113,548369,양식>햄버거,NaN,02-471-0332,http://www.burgerking.co.kr,강동구,길동
111,111,<b>버거킹</b> 둔촌GS FS점,서울특별시 강동구 성내동 439-16,서울특별시 강동구 양재대로 1323,323689,547285,양식>햄버거,NaN,NaN,NaN,강동구,성내동
112,112,<b>버거킹</b> 명일SK점,서울특별시 강동구 명일동 305-51,서울특별시 강동구 고덕로 168 명일주유소,324569,550603,양식>햄버거,NaN,02-426-0332,NaN,강동구,명일동


### 2. 버거킹 갯수만 담긴 데이터 만들기

burgerking_data를 dong을 기준으로 groupby 시켜준 후 

새로운 new_burgerking으로 지정해 줍니다.

new_burgerking에 동마다 count 한 버거킹 갯수를 추가해줍니다.

이제 new_burgerking은 버거킹 갯수를 가진 데이터가 됩니다

In [15]:
new_burgerking = burgerking_data.groupby("dong")
new_burgerking = pd.DataFrame(new_burgerking["address"].count())
new_burgerking.columns = ["burgerking_count"]
new_burgerking

,burgerking_count
dong,
가산동,3
가양동,1
갈현동,1
개봉동,1
거여동,1
...,...
화곡동,1
화양동,1
황학동,2


### 3. korea_data와  new_burgerking 합치기

이제 버거킹 갯수가 담긴 new_burgerking 위의 korea_data를 합칩니다.
기존 korea_data에 버거킹 갯수가 합쳐진 데이터프레임과 합쳐집니다.

In [16]:
korea_data_count = pd.merge(korea_data_count,new_burgerking,on="dong" ,how="left").fillna(0)
korea_data_count["burgerking_count"] = korea_data_count["burgerking_count"].apply(lambda x:int(x))
korea_data_count

,address,gu,dong,mac_count,lot_count,burgerking_count
0,서울특별시,,,0,0,0
1,서울특별시 종로구,,,0,0,0
2,서울특별시 종로구 청운동,종로구,청운동,0,0,0
3,서울특별시 종로구 신교동,종로구,신교동,0,0,0
4,서울특별시 종로구 궁정동,종로구,궁정동,0,0,0
...,...,...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,1,0,0
489,서울특별시 강동구 암사동,강동구,암사동,0,1,0
490,서울특별시 강동구 성내동,강동구,성내동,1,1,3
491,서울특별시 강동구 천호동,강동구,천호동,1,1,0


### 4. 버거킹 갯수를 점수로 환산한 데이터 만들기

new_burgerking 데이터를 바탕으로 버거킹 점수가 담긴 데이터를 만듭니다.

이제 버거킹 점수가 담긴 데이터는 new_burgerking_score 입니다

In [17]:
max_burgerking = new_burgerking['burgerking_count'].max()
new_burgerking_score = new_burgerking.loc[:, ('burgerking_count')]
new_burgerking_score= pd.DataFrame(new_burgerking_score) / max_burgerking * 10
new_burgerking_score.columns = ['burgerking_score']
new_burgerking_score

,burgerking_score
dong,
가산동,10.000000
가양동,3.333333
갈현동,3.333333
개봉동,3.333333
거여동,3.333333
...,...
화곡동,3.333333
화양동,3.333333
황학동,6.666667


### 5. new_burgerking_score와 korea_data_score 합치기

korea_data_score 에 버거킹 점수를 합쳐서 

새로운 korea_data_score 데이터를 갱신합니다.

In [18]:
korea_data_score = pd.merge(korea_data_score,new_burgerking_score,on="dong" ,how="left").fillna(0)
korea_data_score

,address,gu,dong,mac_score,lot_score,burgerking_score
0,서울특별시,,,0.000000,0.00,0.0
1,서울특별시 종로구,,,0.000000,0.00,0.0
2,서울특별시 종로구 청운동,종로구,청운동,0.000000,0.00,0.0
3,서울특별시 종로구 신교동,종로구,신교동,0.000000,0.00,0.0
4,서울특별시 종로구 궁정동,종로구,궁정동,0.000000,0.00,0.0
...,...,...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,3.333333,0.00,0.0
489,서울특별시 강동구 암사동,강동구,암사동,0.000000,1.25,0.0
490,서울특별시 강동구 성내동,강동구,성내동,3.333333,1.25,10.0
491,서울특별시 강동구 천호동,강동구,천호동,3.333333,1.25,0.0


### 1. 기존 데이터 불러오기
kfc.txt 파일을 불러옵니다.

이제 법정동 주소는 korea_data, kfc는 kfc_data 입니다.

In [ ]:
# kfc_data = pd.read_csv('./패스트푸드점/케이에프씨.txt')
# kfc_data['gu'] = kfc_data.apply(lambda x:x["address"].split()[1], axis=1)
# kfc_data['dong'] = kfc_data.apply(lambda x:x["address"].split()[2], axis=1)
# kfc_data

### 2. KFC 갯수만 담긴 데이터 만들기

kfc_data를 dong을 기준으로 groupby 시켜준 후 

새로운 new_kfc으로 지정해 줍니다.

new_kfc에 동마다 count 한 kfc 갯수를 추가해줍니다.

이제 new_kfc는 kfc 갯수를 가진 데이터가 됩니다

In [ ]:
# new_kfc = kfc_data.groupby("dong")
# new_kfc = pd.DataFrame(new_kfc["address"].count())
# new_kfc.columns = ["kfc_count"]
# new_kfc

### 3. korea_data와  new_kfc 합치기

이제 kfc 갯수가 담긴 new_kfc 위의 korea_data를 합칩니다.
기존 korea_data에 kfc 갯수가 합쳐진 데이터프레임과 합쳐집니다.

In [ ]:
# korea_data_count = pd.merge(korea_data_count,new_kfc,on="dong" ,how="left").fillna(0)
# korea_data_count["kfc_count"] = korea_data_count["kfc_count"].apply(lambda x:int(x))
# korea_data_count

### 4. kfc 갯수를 점수로 환산한 데이터 만들기

new_kfc 데이터를 바탕으로 kfc 점수가 담긴 데이터를 만듭니다.

이제 kfc 점수가 담긴 데이터는 new_kfc_score 입니다

In [ ]:
# max_kfc = new_kfc['kfc_count'].max()
# new_kfc_score = new_kfc.loc[:, ('kfc_count')]
# new_kfc_score= pd.DataFrame(new_kfc_score) / max_kfc * 10
# new_kfc_score.columns = ['kfc_score']
# new_kfc_score

### 5. new_kfc_score와 korea_data_score 합치기

korea_data_score 에 kfc 점수를 합쳐서 

새로운 korea_data_score 데이터를 갱신합니다.

In [ ]:
# korea_data_score = pd.merge(korea_data_score,new_kfc_score,on="dong" ,how="left").fillna(0)
# korea_data_score

### 1. 기존 데이터 불러오기
미니스톱.txt 파일을 불러옵니다.

이제 법정동 주소는 korea_data, 미니스톱은 ministop_data 입니다.

In [ ]:
# ministop_data = pd.read_csv('./편의점/미니스톱.txt')
# ministop_data['gu'] = ministop_data.apply(lambda x:x["address"].split()[1], axis=1)
# ministop_data['dong'] = ministop_data.apply(lambda x:x["address"].split()[2], axis=1)
# ministop_data

### 2. 미니스톱 갯수만 담긴 데이터 만들기

ministop_data를 dong을 기준으로 groupby 시켜준 후 

새로운 new_ministop으로 지정해 줍니다.

new_ministop에 동마다 count 한 ministop 갯수를 추가해줍니다.

이제 new_ministop는 ministop 갯수를 가진 데이터가 됩니다

In [ ]:
# new_ministop = ministop_data.groupby("dong")
# new_ministop = pd.DataFrame(new_ministop["address"].count())
# new_ministop.columns = ["ministop_count"]
# new_ministop

### 3. korea_data와  new_ministop 합치기

이제 ministop 갯수가 담긴 new_ministop 위의 korea_data를 합칩니다.
기존 korea_data에 ministop 갯수가 합쳐진 데이터프레임과 합쳐집니다.

In [ ]:
# korea_data_count = pd.merge(korea_data_count,new_ministop,on="dong" ,how="left").fillna(0)
# korea_data_count["ministop_count"] = korea_data_count["ministop_count"].apply(lambda x:int(x))
# korea_data_count

### 4. ministop 갯수를 점수로 환산한 데이터 만들기

new_ministop 데이터를 바탕으로 ministop 점수가 담긴 데이터를 만듭니다.

이제 ministop 점수가 담긴 데이터는 new_ministop_score 입니다

In [ ]:
# max_ministop = new_ministop['ministop_count'].max()
# new_ministop_score = new_ministop.loc[:, ('ministop_count')]
# new_ministop_score= pd.DataFrame(new_ministop_score) / max_ministop * 10
# new_ministop_score.columns = ['ministop_score']
# new_ministop_score

### 5. new_ministop_score와 korea_data_score 합치기

korea_data_score 에 ministop 점수를 합쳐서 

새로운 korea_data_score 데이터를 갱신합니다.

In [ ]:
# korea_data_score = pd.merge(korea_data_score,new_ministop_score,on="dong" ,how="left").fillna(0)
# korea_data_score

### 나머지 데이터 병합

### 1. 나머지 데이터 불러오기

In [19]:
seveneleven_data = pd.read_csv('./편의점/세븐일레븐.txt')
seveneleven_data['gu'] = seveneleven_data.apply(lambda x:x["address"].split()[1], axis=1)
seveneleven_data['dong'] = seveneleven_data.apply(lambda x:x["address"].split()[2], axis=1)

cu_data = pd.read_csv('./편의점/씨유.txt')
cu_data['gu'] = cu_data.apply(lambda x:x["address"].split()[1], axis=1)
cu_data['dong'] = cu_data.apply(lambda x:x["address"].split()[2], axis=1)

gs25_data = pd.read_csv('./편의점/지에스25.txt')
gs25_data['gu'] = gs25_data.apply(lambda x:x["address"].split()[1], axis=1)
gs25_data['dong'] = gs25_data.apply(lambda x:x["address"].split()[2], axis=1)

starbucks_data = pd.read_csv('./카페/스타벅스.txt')
starbucks_data['gu'] = starbucks_data.apply(lambda x:x["address"].split()[1], axis=1)
starbucks_data['dong'] = starbucks_data.apply(lambda x:x["address"].split()[2], axis=1)

lottecinema_data = pd.read_csv('./영화관/롯데시네마.txt')
lottecinema_data['gu'] = lottecinema_data.apply(lambda x:x["address"].split()[1], axis=1)
lottecinema_data['dong'] = lottecinema_data.apply(lambda x:x["address"].split()[2], axis=1)

megabox_data = pd.read_csv('./영화관/메가박스.txt')
megabox_data['gu'] = megabox_data.apply(lambda x:x["address"].split()[1], axis=1)
megabox_data['dong'] = megabox_data.apply(lambda x:x["address"].split()[2], axis=1)

cgv_data = pd.read_csv('./영화관/씨지브이.txt')
cgv_data['gu'] = cgv_data.apply(lambda x:x["address"].split()[1], axis=1)
cgv_data['dong'] = cgv_data.apply(lambda x:x["address"].split()[2], axis=1)

### 2. 갯수만 담긴 데이터 만들기

data를 dong을 기준으로 groupby 시켜준 후 

새로운 new_ooo로 지정해 줍니다.

new_ooo에 동마다 count 한 갯수를 추가해줍니다.

이제 new_ooo는 갯수를 가진 데이터가 됩니다

In [20]:
new_seveneleven = seveneleven_data.groupby("dong")
new_seveneleven = pd.DataFrame(new_seveneleven["address"].count())
new_seveneleven.columns = ["seveneleven_count"]


new_cu = cu_data.groupby("dong")
new_cu = pd.DataFrame(new_cu["address"].count())
new_cu.columns = ["cu_count"]


new_gs25 = gs25_data.groupby("dong")
new_gs25 = pd.DataFrame(new_gs25["address"].count())
new_gs25.columns = ["gs25_count"]


new_starbucks = starbucks_data.groupby("dong")
new_starbucks = pd.DataFrame(new_starbucks["address"].count())
new_starbucks.columns = ["starbucks_count"]


new_lottecinema = lottecinema_data.groupby("dong")
new_lottecinema = pd.DataFrame(new_lottecinema["address"].count())
new_lottecinema.columns = ["lottecinema_count"]


new_megabox = megabox_data.groupby("dong")
new_megabox = pd.DataFrame(new_megabox["address"].count())
new_megabox.columns = ["megabox_count"]


new_cgv = cgv_data.groupby("dong")
new_cgv = pd.DataFrame(new_cgv["address"].count())
new_cgv.columns = ["cgv_count"]

### 3. korea_data와  new_ooo 합치기

이제 갯수가 담긴 new_ooo 위의 korea_data를 합칩니다.
기존 korea_data에 갯수가 합쳐진 데이터프레임과 합쳐집니다.

In [21]:
korea_data_count = pd.merge(korea_data_count,new_seveneleven,on="dong" ,how="left").fillna(0)
korea_data_count["seveneleven_count"] = korea_data_count["seveneleven_count"].apply(lambda x:int(x))

korea_data_count = pd.merge(korea_data_count,new_cu,on="dong" ,how="left").fillna(0)
korea_data_count["cu_count"] = korea_data_count["cu_count"].apply(lambda x:int(x))

korea_data_count = pd.merge(korea_data_count,new_gs25,on="dong" ,how="left").fillna(0)
korea_data_count["gs25_count"] = korea_data_count["gs25_count"].apply(lambda x:int(x))

korea_data_count = pd.merge(korea_data_count,new_starbucks,on="dong" ,how="left").fillna(0)
korea_data_count["starbucks_count"] = korea_data_count["starbucks_count"].apply(lambda x:int(x))

korea_data_count = pd.merge(korea_data_count,new_lottecinema,on="dong" ,how="left").fillna(0)
korea_data_count["lottecinema_count"] = korea_data_count["lottecinema_count"].apply(lambda x:int(x))

korea_data_count = pd.merge(korea_data_count,new_megabox,on="dong" ,how="left").fillna(0)
korea_data_count["megabox_count"] = korea_data_count["megabox_count"].apply(lambda x:int(x))

korea_data_count = pd.merge(korea_data_count,new_cgv,on="dong" ,how="left").fillna(0)
korea_data_count["cgv_count"] = korea_data_count["cgv_count"].apply(lambda x:int(x))

In [ ]:
korea_data_count

### 4. 갯수를 점수로 환산한 데이터 만들기

new_ooo 데이터를 바탕으로 점수가 담긴 데이터를 만듭니다.

이제 점수가 담긴 데이터는 new_ooo_score 입니다

In [22]:
max_seveneleven = new_seveneleven['seveneleven_count'].max()
new_seveneleven_score = new_seveneleven.loc[:, ('seveneleven_count')]
new_seveneleven_score= pd.DataFrame(new_seveneleven_score) / max_seveneleven * 10
new_seveneleven_score.columns = ['seveneleven_score']

max_cu = new_cu['cu_count'].max()
new_cu_score = new_cu.loc[:, ('cu_count')]
new_cu_score= pd.DataFrame(new_cu_score) / max_cu * 10
new_cu_score.columns = ['cu_score']

max_gs25 = new_gs25['gs25_count'].max()
new_gs25_score = new_gs25.loc[:, ('gs25_count')]
new_gs25_score= pd.DataFrame(new_gs25_score) / max_gs25 * 10
new_gs25_score.columns = ['gs25_score']

max_starbucks = new_starbucks['starbucks_count'].max()
new_starbucks_score = new_starbucks.loc[:, ('starbucks_count')]
new_starbucks_score= pd.DataFrame(new_starbucks_score) / max_starbucks * 10
new_starbucks_score.columns = ['starbucks_score']

max_lottecinema = new_lottecinema['lottecinema_count'].max()
new_lottecinema_score = new_lottecinema.loc[:, ('lottecinema_count')]
new_lottecinema_score= pd.DataFrame(new_lottecinema_score) / max_lottecinema * 10
new_lottecinema_score.columns = ['lottecinema_score']

max_megabox = new_megabox['megabox_count'].max()
new_megabox_score = new_megabox.loc[:, ('megabox_count')]
new_megabox_score= pd.DataFrame(new_megabox_score) / max_megabox * 10
new_megabox_score.columns = ['megabox_score']

max_cgv = new_cgv['cgv_count'].max()
new_cgv_score = new_cgv.loc[:, ('cgv_count')]
new_cgv_score= pd.DataFrame(new_cgv_score) / max_cgv * 10
new_cgv_score.columns = ['cgv_score']

### 5. new_ooo_score와 korea_data_score 합치기

korea_data_score 에 점수를 합쳐서 

새로운 korea_data_score 데이터를 갱신합니다.

In [23]:
korea_data_score = pd.merge(korea_data_score,new_seveneleven_score,on="dong" ,how="left").fillna(0)

korea_data_score = pd.merge(korea_data_score,new_cu_score,on="dong" ,how="left").fillna(0)

korea_data_score = pd.merge(korea_data_score,new_gs25_score,on="dong" ,how="left").fillna(0)

korea_data_score = pd.merge(korea_data_score,new_starbucks_score,on="dong" ,how="left").fillna(0)

korea_data_score = pd.merge(korea_data_score,new_lottecinema_score,on="dong" ,how="left").fillna(0)

korea_data_score = pd.merge(korea_data_score,new_megabox_score,on="dong" ,how="left").fillna(0)

korea_data_score = pd.merge(korea_data_score,new_cgv_score,on="dong" ,how="left").fillna(0)

In [24]:
korea_data_score

,address,gu,dong,mac_score,lot_score,burgerking_score,seveneleven_score,cu_score,gs25_score,starbucks_score,lottecinema_score,megabox_score,cgv_score
0,서울특별시,,,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0
1,서울특별시 종로구,,,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0
2,서울특별시 종로구 청운동,종로구,청운동,0.000000,0.00,0.0,0.000000,0.000000,0.125,0.000000,0.0,0.0,0.0
3,서울특별시 종로구 신교동,종로구,신교동,0.000000,0.00,0.0,0.000000,0.133333,0.000,0.000000,0.0,0.0,0.0
4,서울특별시 종로구 궁정동,종로구,궁정동,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,3.333333,0.00,0.0,0.851064,0.666667,0.875,0.000000,0.0,0.0,0.0
489,서울특별시 강동구 암사동,강동구,암사동,0.000000,1.25,0.0,1.063830,1.733333,1.750,0.434783,0.0,0.0,0.0
490,서울특별시 강동구 성내동,강동구,성내동,3.333333,1.25,10.0,2.553191,2.666667,3.000,1.739130,5.0,10.0,0.0
491,서울특별시 강동구 천호동,강동구,천호동,3.333333,1.25,0.0,4.680851,2.933333,3.125,0.869565,0.0,0.0,5.0


In [25]:
korea_data_score['total_score'] = korea_data_score.sum(axis=1)

In [26]:
max_korea_data_score = korea_data_score['total_score'].max()

max_korea_data_score

71.81666666666666

In [27]:
korea_data_score

,address,gu,dong,mac_score,lot_score,burgerking_score,seveneleven_score,cu_score,gs25_score,starbucks_score,lottecinema_score,megabox_score,cgv_score,total_score
0,서울특별시,,,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0,0.000000
1,서울특별시 종로구,,,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0,0.000000
2,서울특별시 종로구 청운동,종로구,청운동,0.000000,0.00,0.0,0.000000,0.000000,0.125,0.000000,0.0,0.0,0.0,0.125000
3,서울특별시 종로구 신교동,종로구,신교동,0.000000,0.00,0.0,0.000000,0.133333,0.000,0.000000,0.0,0.0,0.0,0.133333
4,서울특별시 종로구 궁정동,종로구,궁정동,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,3.333333,0.00,0.0,0.851064,0.666667,0.875,0.000000,0.0,0.0,0.0,5.726064
489,서울특별시 강동구 암사동,강동구,암사동,0.000000,1.25,0.0,1.063830,1.733333,1.750,0.434783,0.0,0.0,0.0,6.231946
490,서울특별시 강동구 성내동,강동구,성내동,3.333333,1.25,10.0,2.553191,2.666667,3.000,1.739130,5.0,10.0,0.0,39.542322
491,서울특별시 강동구 천호동,강동구,천호동,3.333333,1.25,0.0,4.680851,2.933333,3.125,0.869565,0.0,0.0,5.0,21.192083


In [28]:
korea_data_count['total_count'] = korea_data_count.sum(axis=1)

In [29]:
korea_data_count

,address,gu,dong,mac_count,lot_count,burgerking_count,seveneleven_count,cu_count,gs25_count,starbucks_count,lottecinema_count,megabox_count,cgv_count,total_count
0,서울특별시,,,0,0,0,0,0,0,0,0,0,0,0
1,서울특별시 종로구,,,0,0,0,0,0,0,0,0,0,0,0
2,서울특별시 종로구 청운동,종로구,청운동,0,0,0,0,0,1,0,0,0,0,1
3,서울특별시 종로구 신교동,종로구,신교동,0,0,0,0,1,0,0,0,0,0,1
4,서울특별시 종로구 궁정동,종로구,궁정동,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,1,0,0,4,5,7,0,0,0,0,17
489,서울특별시 강동구 암사동,강동구,암사동,0,1,0,5,13,14,1,0,0,0,34
490,서울특별시 강동구 성내동,강동구,성내동,1,1,3,12,20,24,4,1,1,0,67
491,서울특별시 강동구 천호동,강동구,천호동,1,1,0,22,22,25,2,0,0,1,74


In [30]:
korea_data_score.to_csv("편의시설 점수.txt", mode="w")

In [31]:
korea_data_count.to_csv("편의시설 개수.txt", mode="w")

In [32]:
oil_data = pd.read_csv('../data_gdr/2-4. 유가 정보(10월 4주차) 평균 가격(최종).txt')

In [33]:
oil_data

,dong,high_oil,oil,oil2
0,가락동,1852.715000,1615.5700,1486.03500
1,가리봉동,NaN,1579.0000,1377.00000
2,가산동,NaN,1496.4300,1346.43000
3,가양동,1742.000000,1543.6075,1395.17750
4,갈월동,2249.000000,1999.0000,1899.00000
...,...,...,...,...
184,화곡동,1685.906667,1498.8575,1348.76625
185,화양동,1985.000000,1595.0000,1469.00000
186,후암동,2297.000000,1955.1400,1865.14000
187,휘경동,1775.000000,1578.5000,1399.03750


In [34]:
korea_data_score = pd.merge(korea_data_score,oil_data,on="dong" ,how="left").fillna(0)

In [35]:
korea_data_score

,address,gu,dong,mac_score,lot_score,burgerking_score,seveneleven_score,cu_score,gs25_score,starbucks_score,lottecinema_score,megabox_score,cgv_score,total_score,high_oil,oil,oil2
0,서울특별시,,,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.000
1,서울특별시 종로구,,,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.000
2,서울특별시 종로구 청운동,종로구,청운동,0.000000,0.00,0.0,0.000000,0.000000,0.125,0.000000,0.0,0.0,0.0,0.125000,0.00,0.00,0.000
3,서울특별시 종로구 신교동,종로구,신교동,0.000000,0.00,0.0,0.000000,0.133333,0.000,0.000000,0.0,0.0,0.0,0.133333,0.00,0.00,0.000
4,서울특별시 종로구 궁정동,종로구,궁정동,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,3.333333,0.00,0.0,0.851064,0.666667,0.875,0.000000,0.0,0.0,0.0,5.726064,1887.07,1637.07,1487.070
489,서울특별시 강동구 암사동,강동구,암사동,0.000000,1.25,0.0,1.063830,1.733333,1.750,0.434783,0.0,0.0,0.0,6.231946,1912.07,1643.38,1482.620
490,서울특별시 강동구 성내동,강동구,성내동,3.333333,1.25,10.0,2.553191,2.666667,3.000,1.739130,5.0,10.0,0.0,39.542322,1796.00,1603.75,1467.965
491,서울특별시 강동구 천호동,강동구,천호동,3.333333,1.25,0.0,4.680851,2.933333,3.125,0.869565,0.0,0.0,5.0,21.192083,0.00,1628.50,1483.000


In [36]:
crime_data = pd.read_csv('../data_nenemttin/서울시구별범죄현황.txt', encoding='CP949', sep=' ')

In [37]:
crime_data

,gu,total,catch,percent
0,종로구,3690,3913,106.0
1,중구,4030,2679,66.5
2,용산구,3411,2543,74.6
3,성동구,2457,1787,72.7
4,광진구,3915,2789,71.2
5,동대문구,3680,2968,80.7
6,중랑구,4288,3222,75.1
7,성북구,3042,2277,74.9
8,강북구,3437,2743,79.8
9,도봉구,2249,1676,74.5


In [38]:
korea_data_score = pd.merge(korea_data_score,crime_data,on="gu" ,how="left").fillna(0)
korea_data_score

,address,gu,dong,mac_score,lot_score,burgerking_score,seveneleven_score,cu_score,gs25_score,starbucks_score,lottecinema_score,megabox_score,cgv_score,total_score,high_oil,oil,oil2,total,catch,percent
0,서울특별시,,,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.000,0.0,0.0,0.0
1,서울특별시 종로구,,,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.000,0.0,0.0,0.0
2,서울특별시 종로구 청운동,종로구,청운동,0.000000,0.00,0.0,0.000000,0.000000,0.125,0.000000,0.0,0.0,0.0,0.125000,0.00,0.00,0.000,3690.0,3913.0,106.0
3,서울특별시 종로구 신교동,종로구,신교동,0.000000,0.00,0.0,0.000000,0.133333,0.000,0.000000,0.0,0.0,0.0,0.133333,0.00,0.00,0.000,3690.0,3913.0,106.0
4,서울특별시 종로구 궁정동,종로구,궁정동,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.000,3690.0,3913.0,106.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,3.333333,0.00,0.0,0.851064,0.666667,0.875,0.000000,0.0,0.0,0.0,5.726064,1887.07,1637.07,1487.070,3919.0,2789.0,71.2
489,서울특별시 강동구 암사동,강동구,암사동,0.000000,1.25,0.0,1.063830,1.733333,1.750,0.434783,0.0,0.0,0.0,6.231946,1912.07,1643.38,1482.620,3919.0,2789.0,71.2
490,서울특별시 강동구 성내동,강동구,성내동,3.333333,1.25,10.0,2.553191,2.666667,3.000,1.739130,5.0,10.0,0.0,39.542322,1796.00,1603.75,1467.965,3919.0,2789.0,71.2
491,서울특별시 강동구 천호동,강동구,천호동,3.333333,1.25,0.0,4.680851,2.933333,3.125,0.869565,0.0,0.0,5.0,21.192083,0.00,1628.50,1483.000,3919.0,2789.0,71.2


In [39]:
korea_data_score["total"] = korea_data_score["total"].apply(lambda x:int(x))
korea_data_score["catch"] = korea_data_score["catch"].apply(lambda x:int(x))

In [40]:
korea_data_score

,address,gu,dong,mac_score,lot_score,burgerking_score,seveneleven_score,cu_score,gs25_score,starbucks_score,lottecinema_score,megabox_score,cgv_score,total_score,high_oil,oil,oil2,total,catch,percent
0,서울특별시,,,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.000,0,0,0.0
1,서울특별시 종로구,,,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.000,0,0,0.0
2,서울특별시 종로구 청운동,종로구,청운동,0.000000,0.00,0.0,0.000000,0.000000,0.125,0.000000,0.0,0.0,0.0,0.125000,0.00,0.00,0.000,3690,3913,106.0
3,서울특별시 종로구 신교동,종로구,신교동,0.000000,0.00,0.0,0.000000,0.133333,0.000,0.000000,0.0,0.0,0.0,0.133333,0.00,0.00,0.000,3690,3913,106.0
4,서울특별시 종로구 궁정동,종로구,궁정동,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.0,0.000000,0.00,0.00,0.000,3690,3913,106.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,3.333333,0.00,0.0,0.851064,0.666667,0.875,0.000000,0.0,0.0,0.0,5.726064,1887.07,1637.07,1487.070,3919,2789,71.2
489,서울특별시 강동구 암사동,강동구,암사동,0.000000,1.25,0.0,1.063830,1.733333,1.750,0.434783,0.0,0.0,0.0,6.231946,1912.07,1643.38,1482.620,3919,2789,71.2
490,서울특별시 강동구 성내동,강동구,성내동,3.333333,1.25,10.0,2.553191,2.666667,3.000,1.739130,5.0,10.0,0.0,39.542322,1796.00,1603.75,1467.965,3919,2789,71.2
491,서울특별시 강동구 천호동,강동구,천호동,3.333333,1.25,0.0,4.680851,2.933333,3.125,0.869565,0.0,0.0,5.0,21.192083,0.00,1628.50,1483.000,3919,2789,71.2


In [41]:
final_data = pd.merge(korea_data_score, korea_data_count, on=["dong","address", "gu"], how="left").fillna(0)

In [42]:
final_data

,address,gu,dong,mac_score,lot_score,burgerking_score,seveneleven_score,cu_score,gs25_score,starbucks_score,...,lot_count,burgerking_count,seveneleven_count,cu_count,gs25_count,starbucks_count,lottecinema_count,megabox_count,cgv_count,total_count
0,서울특별시,,,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,서울특별시 종로구,,,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,서울특별시 종로구 청운동,종로구,청운동,0.000000,0.00,0.0,0.000000,0.000000,0.125,0.000000,...,0,0,0,0,1,0,0,0,0,1
3,서울특별시 종로구 신교동,종로구,신교동,0.000000,0.00,0.0,0.000000,0.133333,0.000,0.000000,...,0,0,0,1,0,0,0,0,0,1
4,서울특별시 종로구 궁정동,종로구,궁정동,0.000000,0.00,0.0,0.000000,0.000000,0.000,0.000000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,3.333333,0.00,0.0,0.851064,0.666667,0.875,0.000000,...,0,0,4,5,7,0,0,0,0,17
489,서울특별시 강동구 암사동,강동구,암사동,0.000000,1.25,0.0,1.063830,1.733333,1.750,0.434783,...,1,0,5,13,14,1,0,0,0,34
490,서울특별시 강동구 성내동,강동구,성내동,3.333333,1.25,10.0,2.553191,2.666667,3.000,1.739130,...,1,3,12,20,24,4,1,1,0,67
491,서울특별시 강동구 천호동,강동구,천호동,3.333333,1.25,0.0,4.680851,2.933333,3.125,0.869565,...,1,0,22,22,25,2,0,0,1,74


In [45]:
final_data['mac_score'] = final_data['mac_score'].round(2)
final_data['lot_score'] = final_data['lot_score'].round(2)
final_data['burgerking_score'] = final_data['burgerking_score'].round(3)
final_data['seveneleven_score'] = final_data['seveneleven_score'].round(2)
final_data['cu_score'] = final_data['cu_score'].round(2)
final_data['gs25_score'] = final_data['gs25_score'].round(2)
final_data['starbucks_score'] = final_data['starbucks_score'].round(2)
final_data['lottecinema_score'] = final_data['lottecinema_score'].round(2)
final_data['megabox_score'] = final_data['megabox_score'].round(2)
final_data['cgv_score'] = final_data['cgv_score'].round(2)
final_data['total_score'] = final_data['total_score'].round(2)
final_data['high_oil'] = final_data['high_oil'].round(2)
final_data['oil'] = final_data['oil'].round(2)
final_data['oil2'] = final_data['oil2'].round(2)
final_data['percent'] = final_data['percent'].round(2)

In [46]:
final_data

,address,gu,dong,mac_score,lot_score,burgerking_score,seveneleven_score,cu_score,gs25_score,starbucks_score,...,lot_count,burgerking_count,seveneleven_count,cu_count,gs25_count,starbucks_count,lottecinema_count,megabox_count,cgv_count,total_count
0,서울특별시,,,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
1,서울특별시 종로구,,,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
2,서울특별시 종로구 청운동,종로구,청운동,0.00,0.00,0.0,0.00,0.00,0.12,0.00,...,0,0,0,0,1,0,0,0,0,1
3,서울특별시 종로구 신교동,종로구,신교동,0.00,0.00,0.0,0.00,0.13,0.00,0.00,...,0,0,0,1,0,0,0,0,0,1
4,서울특별시 종로구 궁정동,종로구,궁정동,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,서울특별시 강동구 둔촌동,강동구,둔촌동,3.33,0.00,0.0,0.85,0.67,0.88,0.00,...,0,0,4,5,7,0,0,0,0,17
489,서울특별시 강동구 암사동,강동구,암사동,0.00,1.25,0.0,1.06,1.73,1.75,0.43,...,1,0,5,13,14,1,0,0,0,34
490,서울특별시 강동구 성내동,강동구,성내동,3.33,1.25,10.0,2.55,2.67,3.00,1.74,...,1,3,12,20,24,4,1,1,0,67
491,서울특별시 강동구 천호동,강동구,천호동,3.33,1.25,0.0,4.68,2.93,3.12,0.87,...,1,0,22,22,25,2,0,0,1,74


In [47]:
final_data.to_csv("최종 결과.txt", mode="w")

In [ ]:
# k = 0
# for i in mcdonald_data["gu"].value_counts(dropna=False).items():
#     print(i)
#     k += i[1]
# print(k)

In [ ]:
# k = 0
# for i in mcdonald_data["dong"].value_counts(dropna=False).items():
#     print(i)
#     k += i[1]
# print(k)

In [ ]:
# new_lotte = lotteria_data.groupby("dong")
# new_lotte = pd.DataFrame(new_lotte["address"].count())
# new_lotte.columns = ['롯데리아']
# new_korea_data = pd.merge(new_korea_data,new_lotte,on="dong" ,how="left").fillna(0)
# new_korea_data

In [ ]:
# new_burgerking = burgerking_data.groupby("dong")
# new_burgerking = pd.DataFrame(new_burgerking["address"].count())
# new_burgerking.columns = ['버거킹']
# new_korea_data = pd.merge(new_korea_data,new_burgerking,on="dong" ,how="left").fillna(0)
# new_korea_data

In [ ]:
# new_kfc = kfc_data.groupby("dong")
# new_kfc = pd.DataFrame(new_kfc["address"].count())
# new_kfc.columns = ['kfc']
# new_korea_data = pd.merge(new_korea_data,new_kfc,on="dong" ,how="left").fillna(0)
# new_korea_data

In [ ]:
# new_korea_data["맥도날드"] = new_korea_data["맥도날드"].apply(lambda x:int(x))
# new_korea_data["롯데리아"] = new_korea_data["롯데리아"].apply(lambda x:int(x))
# new_korea_data["버거킹"] = new_korea_data["버거킹"].apply(lambda x:int(x))
# new_korea_data["kfc"] = new_korea_data["kfc"].apply(lambda x:int(x))

In [ ]:
# new_korea_data

In [ ]:
# new_korea_data.to_csv('C:/Users/multicampus/yogeuncheo/data/data_adg/프렌차이즈 개수.txt', header=True, index=False, encoding="UTF-8")